Version notes:
- need to remove bonds which do not have returns on consecutive months (or go back and see why this is) (Example: issue id 5)

# ML Analysis

In [1]:
import pandas as pd
import numpy as np

## Data
Data are from 'data_cleaning.ipynb', which uses a subset of merged TRACE and Mergent FISD daily bond data. It follows the sample exclusion criteria of Bai, Bali & Wen (2019) and aggregates returns to monthly based on their methodologies.

In [135]:
data = pd.read_csv('monthly_data.csv', index_col=0)

In [7]:
data.head()

,date,issue_id,cusip_id,volume,issuer_id,prospectus_issuer_name,maturity,offering_amt,interest_frequency,cusip,...,rating_spr,rating_mdy,amount_outstanding,tau,age,ytm,duration,tr_dirty_price,tr_ytm,return
0,2002-08-31,98694,010392DN5,25000.0,86.0,ALABAMA PWR CO,2004-08-15,250000.0,2.0,010392DN5,...,A,A2,250000.0,1.972603,3.038356,3.672086,1.872172,109.77133,2.142072,0.031350
1,2002-08-31,99961,010392DP0,15000.0,86.0,ALABAMA PWR CO,2007-10-01,200000.0,2.0,010392DP0,...,A,A2,200000.0,5.101370,2.931507,4.540522,4.299664,120.46437,3.339660,0.065160
2,2002-08-31,144179,00079FAW2,20000.0,37284.0,ABN AMRO BK N V,2003-12-19,4500.0,2.0,00079FAW2,...,AA,Aa2,4500.0,1.315068,0.200000,12.814093,1.230101,115.34654,1.844255,0.130164
3,2002-08-31,97686,013104AE4,215000.0,93.0,ALBERTSONS INC,2009-08-01,350000.0,2.0,013104AE4,...,BBB+,Baa1,350000.0,6.936986,3.098630,5.601379,5.647303,119.27208,3.834566,0.055352
4,2002-08-31,102864,029050AB7,2000000.0,33708.0,AMERICAN PLUMBING & MECHANICAL,2008-10-15,125000.0,2.0,029050AB7,...,B-,B3,125000.0,6.142466,2.734246,29.812586,3.527641,148.25096,3.569999,-0.324551


Following Moritz and Zimmerman (2016) (MZ), the goal is to estimate the expected return of stock i in time period t+1, conditional on information in period t.

In the context of corporate bonds, our aim will be to predict the return of bond i. Then based on the predicted return for t+1, bonds are sorted into deciles based on the expected return, and the trading strategy goes long the top decile (ie. highest predicted returns) and short the bottom decile. The out-of-sample test is implemented on a rolling basis: the model is estimated every year, using data from the past 5 years. Returns are predicted for the next 12 months, and in each month the decile portfolio strategy is implemented.

Given the bond characteristics, such as credit rating, duration, and yield-to-maturity (which are all known factors to affect returns), three types of return models can be implemented. The first can focus on raw returns, the second could focus on returns in "excess" of these other factors, and the third can incorporate these characteristics and return-based factors simultaneously. The first model ignores potential correlation between the return-based factors and other bond characteristics, however it follows Moritz and Zimmerman's methodology. The second model would involve first orthogonalizing returns to the bond factors (using linear regression), and then predicting the residual returns; this is appealing, but introduces estimation uncertainty (ie. what if the relationship is non-linear). Lastly, the third methodology is the most flexible. For example, it may be optimal to first sort on duration, and then on past returns etc. 


Thus, the outcome variables (ie. to be predicted) is the next month's return, up to the next 11 months. ~~To simplify things, it may be worthwhile to consider a buy-and-hold strategy: estimate the model, predict the next cummulative X month return (say 3 or 12 months), sort stocks, and go long/sort the portfolios with a holding period of X months. This would have significantly less turnover and hence less transaction costs. This could be used as a robustness check.~~ (This is impractical data wise)

The predictor variables are the past 24 months of 1 month returns (ie. the return over month t-k to t-k+1 for k = (1,...24)). The notation used by Moritz and Zimmerman is $ R_{i,t}(k,1) $. Thus the tree-based model effectively estimates a more complicated version of:

$
r_{i,t+1} = \mu_{1t} I(R_{i,t}(k,1) \lt \tau_t) + \mu_{2t} I(R_{i,t}(k,1) \ge \tau_t)
$

Where $\mu_1t$ can be interpreted as the return on a portfolio for which all stocks $i$ satisfy $R_{i,t}(k,1) \lt \tau_t$ in period $t$. Hence instead of simply sorting on a particular decile, stocks are sorted deeply on potentially many different criteria.

In terms of implementation practicicality, the ML model can be estimated akin to the Fama-MacBeth procedure. Using the past 5 years (60 months) of data, predictions can be formed. The estimation is over the time-dimension (in an OLS framework equivalent to averaging the results of 60 cross-sectional regressions), while the prediction is over the cross-sectional dimension (ie. bonds are ranked by highest return in the next cross-section). 

Hence the data should be indexed at the bond-month level (ie. all predictors and outcomes should be aligned to month t for bond i). Thus estimating a model for bond i using data from t-60 to t is equivalent to a pooled cross-section. 

## Summary Stats

In [136]:
# Chronological order, indexes by month-bond
data = data.sort_values(by=['date', 'issue_id'])
data = data.set_index(['date', 'issue_id'])

In [43]:
print('There are', len(data.index.get_level_values(0).unique()), ' unique months in the panel')

There are 234  unique months in the panel


In [52]:
bond_count = {}
for month in data.index.get_level_values(0).unique():
    bond_count[month] = len(data.loc[month])

bonds = pd.DataFrame.from_dict(bond_count, orient='index', columns=['Bond Count'])

In [61]:
print('There are on average ', bonds.describe().loc['mean'][0].astype(int), 
      ' bonds in each monthly cross-section, with as few as ', bonds.describe().loc['min'][0], 
      ' and as many as ', bonds.describe().loc['max'][0])

There are on average  373  bonds in each monthly cross-section, with as few as  190.0  and as many as  594.0


## Outcome-Predictor Organization

Since predictors and outcomes are both returns, the notation will follow from MZ, where predictors will be denoted as R(k,1) (ie. the historical 1 month return for month k), while the outcome will be denoted as r(k,t) (ie. the t-month cummulative return of holding the bond k months ahead). As mentioned previously, the main outcomes will be {r(1,1),...,r(12,1)} ~~as well as {r(1,3),r(1,12)}.~~

In [389]:
df = data.copy()

In [390]:
# Predictors: Past 1-month Returns
for i in range(24):
    df['R('+str(i)+',1)'] = df.groupby(level=1)['return'].shift(i+1)

In [395]:
# Example of predictors
# df.xs(54, level=1).iloc[:,19:]

In [392]:
# Outcomes:
# 12 months of 1-month returns
for i in range(12):
    df['r('+str(i+1)+',1)'] = df.groupby(level=1)['return'].shift(-(i+1))
    
# Buy and hold returns (ChatGPT code that doesnt work)
# df['r(1,3)'] = df.groupby(level=1)['return'].shift(-1).groupby(level=1).rolling(3).apply(lambda x: (1+x).prod()-1).groupby(level=1).shift(-2).reset_index(drop=True)
# df['r(1,12)'] = ((1+df.groupby(level=1)['return'].shift(-1)).groupby(level=1).cumprod() - 1).groupby(level=1).nth[11]

In [388]:
# df.xs(54, level=1).iloc[:,19:]

In [387]:
# group = df.groupby(level=1)['return'].shift(-1).rolling(3).apply(lambda x: (1+x).prod()-1).groupby(level=1).shift(-2)
# # Correct output
# df.xs(54, level=1)['return'].shift(-1).rolling(3).apply(lambda x: (1+x).prod()-1).shift(-2)

In [394]:
df.iloc[:,19:]

return    R(0,1)  R(1,1)  R(2,1)  R(3,1)  R(4,1)  \
date       issue_id                                                       
2002-07-31 38       -0.020465       NaN     NaN     NaN     NaN     NaN   
           54        0.059929       NaN     NaN     NaN     NaN     NaN   
           90        0.070783       NaN     NaN     NaN     NaN     NaN   
           91        0.039458       NaN     NaN     NaN     NaN     NaN   
           92        0.083611       NaN     NaN     NaN     NaN     NaN   
...                       ...       ...     ...     ...     ...     ...   
2021-12-31 1002439   0.025130  0.018604     NaN     NaN     NaN     NaN   
           1005099   0.008477  0.005862     NaN     NaN     NaN     NaN   
           1005163   0.021746  0.008483     NaN     NaN     NaN     NaN   
           1006823   0.014629       NaN     NaN     NaN     NaN     NaN   
           1006831   0.021220       NaN     NaN     NaN     NaN     NaN   

                     R(5,1)  R(6,1)  R(7,1)  R(8,1)  ...    r(3,1)    r(4,1)  \
date       issue_id                                  ...                       
2002-07-31 38           NaN     NaN     NaN     NaN  ...  0.018710  0.698084   
           54           NaN     NaN     NaN     NaN  ... -0.119249  0.778761   
           90           NaN     NaN     NaN     NaN  ...  0.103232  0.146729   
           91           NaN     NaN     NaN     NaN  ...  0.105658  0.149718   
           92           NaN     NaN     NaN     NaN  ...  0.127010  0.099970   
...                     ...     ...     ...     ...  ...       ...       ...   
2021-12-31 1002439      NaN     NaN     NaN     NaN  ...       NaN       NaN   
           1005099      NaN     NaN     NaN     NaN  ...       NaN       NaN   
           1005163      NaN     NaN     NaN     NaN  ...       NaN       NaN   
           1006823      NaN     NaN     NaN     NaN  ...       NaN       NaN   
           1006831      NaN     NaN     NaN     NaN  ...       NaN       NaN   

                       r(5,1)    r(6,1)    r(7,1)    r(8,1)    r(9,1)  \
date       issue_id                                                     
2002-07-31 38        0.037567 -0.184631  0.123870  0.096401  1.486548   
           54        0.067102 -0.228833 -0.059902  0.432816  1.519604   
           90        0.119039  0.056269  0.090864  0.105388  0.088816   
           91        0.129472  0.054685  0.065900  0.113197  0.098663   
           92        0.103820  0.124113  0.042410  0.132478  0.105280   
...                       ...       ...       ...       ...       ...   
2021-12-31 1002439        NaN       NaN       NaN       NaN       NaN   
           1005099        NaN       NaN       NaN       NaN       NaN   
           1005163        NaN       NaN       NaN       NaN       NaN   
           1006823        NaN       NaN       NaN       NaN       NaN   
           1006831        NaN       NaN       NaN       NaN       NaN   

                      r(10,1)   r(11,1)   r(12,1)  
date       issue_id                                
2002-07-31 38        0.362272  0.576254  0.040227  
           54        0.357057  0.503291  0.133504  
           90        0.102074  0.084817  0.033955  
           91        0.107632  0.063911  0.059486  
           92        0.096793  0.060019  0.070597  
...                       ...       ...       ...  
2021-12-31 1002439        NaN       NaN       NaN  
           1005099        NaN       NaN       NaN  
           1005163        NaN       NaN       NaN  
           1006823        NaN       NaN       NaN  
           1006831        NaN       NaN       NaN  

[87369 rows x 37 columns]

## Model Estimation

Several models can be implemented. 

1) OLS
- All 24 past returns can be used as predictors

2) Regularization
- Same predictors as OLS, but can include all combinations of 2 or 3 way interactions
- LASSO seems like an obvious candidate

3) Regression Tree
- Can be used to demonstrate the underlying mechanics and interpretation
- Will likely overfit, but still worthwhile to implement

4) Random Forest
- Follows from MZ

5) XGBoost
6) Neural Network???

Regardless of which model is implemented, the out-of-sample procedure is identical. 

MZ don't explicitly use cross-validation to tune hyperparameters, noting "we construct 200 tree-based conditional portfolio sorts and we use 8 out of 25 regressors in each of them. We have tried other values for the [hyperparameters] but have found that results do not vary much with these choices. We settled on the share of 30 percent of regressors because it is a standard 
recommendation in the random forest literature, and we chose B = 200 because higher values did not have any apparent benefit for the estimation but are more costly in terms of computation."

The out-of-sample procedure is as follows:
1) Estimate the model using data from the past 5 years (60 months)
2) Predict returns for the next 12 months
3) Based on the predictions, for each month, sort bonds into deciles, and go long the top decile and short the bottom decile
4) The trading strategy rebalances monthly based on the rankings from the previous step
5) After 1 year, the model is re-estimated and the process repeats

An alternative strategy, which would reduce monthly turnover, could re-rank bonds based on their average decile over the next 12 months, and then go long/short the top/bottom decile of the second-order sorting. The long/short would be held for the next 12 months, and the strategy would be rebalanced annually. 

In [ ]:
def ols_model():
    '''
    
    '''